# Análise de sentimentos do Twitter e bolsa de valores: uma investigação sobre a correlação entre estes dois sistemas

     Este projeto tem por objetivo investigar se existe correlação entre a opinião pública sobre os papéis da bolsa de valores na rede social Twitter e o retorno das ações na bolsa de valores. Para isto, aplicar-se-á técnicas de processamento de linguagem natural para extração de sentimentos, a análise de sentimentos, e algoritmos de machine learning a fim de averiguar se há tal correlação. Os códigos-fonte foram divididos em 3 arquivos: Geração de dados, análise dos dados e aplicação de machine learning.

## Arquivo 1: Gerador de dados

Este módulo tem por objetivo:

    1. Gerar uma base de dados e salvar em arquivo local;
    2. Persistir a base de dados no MongoDB;

### 1. Preparando Ambiente

#### 1.1 Instalando dependências

In [ ]:
#yahoo finances para recuperar dados da bolsa de valores
!pip install yfinance --upgrade --no-cache-dir

#Scrape tweet ids
!pip install snscrape

#API to retrieve tweets by id
!pip install tweepy

#### 1.2 Importando bibliotecas

In [1]:
import os
#API Yahoo Finance
import yfinance as yf
#API twitter
import tweepy
#Maniputalacao de dados em dataframe
import pandas as pd

### 2. Aquisição da base de dados

#### 2.1 Finanças
Nesta etapa, baixa-se os dados financeiros através do Yahoo Finances e salva-os em uma lista de objetos do tipo dicionário do python.

In [5]:
#setting variable dates
date_start = '2019-01-01'; date_end = '2019-05-31'

# set directory to storage financial files
path= './database/finance/'

# Stock market tickers for downloading
dict_search = {
        'Magazine Luisa SA': 'MGLU3.SA',
        'Petrobras': 'PETR4.SA',
        'B3': 'B3SA3.SA', 
        'IRB Brasil Seguros': 'IRBR3.SA',
}

for key in dict_search:
    df = yf.download(dict_search[key], start=date_start, end=date_end)
    
    #Saving to dataframe in local storage path
    pd.DataFrame(df).to_csv(path+key+'.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


#### 2.2 Twitter

#### snscrape

O comando snscrape funciona, em geral, no prompt de comando:

       snscrape twitter-search "MGLU3 since:2019-01-01 until:2019-05-31" > MGLU3_tweets.txt
tal que, "MGLU3" foi o ticker buscado, "since:2019-01-01" e "until:2019-05-31" definem datas de início e término da busca. Assim, vamos utilizar este comando usando a biblioteca OS para fazer a busca das palavras-chave do twitter.

In [ ]:
search_terms = ['MGLU3', 'PETR4', 'B3SA3', 'IRBR3']
file_names = ['Magalu', 'Petrobras','B3', 'IRB_Brasil_Seguros']
base = 'snscrape twitter-search "'
since = '2019-01-01'
until = '2019-05-31'

for i,ticker in enumerate(search_terms):
    os.system(base+ticker+' --since:'+since+' --until:'+until+'" > ./snscrape/'+file_names[i]+".txt")

Agora temos os arquivos necessários que contém os links dos tweets buscados. Agora, segue a etapa de recuperação dos tweets por meio do tweepy.

#### Tweepy

Conexão com a API:

In [2]:
# Fill the variables with your Twitter developer account credentials 
api_key = ''
api_secret_key = ''
access_token = ''
access_token_secret = ''

# Connecting to API
auth = tweepy.OAuthHandler(api_key, api_secret_key) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Definindo função de busca. Passa-se uma lista de ids e retorna uma lista de objetos do tipo dict python com os tweets.

In [5]:
# Set directory to storage twitter files
twitter_path = './database/twitter/'

search_terms = ['MGLU3', 'PETR4', 'B3SA3', 'IRBR3']
file_names = ['Magalu', 'Petrobras','B3', 'IRB_Brasil_Seguros']

all_tweets = []

def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data_list = []
    for status in list_of_tw_status:
            tweet_elem = {"date":status.created_at,
                          "arroba": status.user.screen_name,
                          "text":status.full_text,
                          "tweet_id": status.id,
                          "favorite":status.favorite_count,
                          "retweet": status.retweet_count
                     }
            empty_data_list.append(tweet_elem)
    return empty_data_list

for index, item in enumerate(search_terms):
    
    #Leitura dos scrapings
    file_path = './snscrape/'+file_names[index]+'.txt'
    tweet_url = pd.read_csv(file_path, index_col= None, header = None, names = ["links"])

    #Extraindo id dos links
    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)
    ids = tweet_url['id'].tolist()
    
    #Baixar tweets de 50 em 50
    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1

    result = []

    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result += fetch_tw(batch)
    
    #Salvando localmente
    #print(result)
    df = pd.DataFrame(result)
    df.to_csv(twitter_path+file_names[index]+'.csv')
    
    #salvar todos os tweets
    all_tweets+= result

### 3) Observações durante as tarefas

#### API Obsoleta GetOldTweets3

Infelizmente, a API GetOldTweets3 já não funciona mais para buscas avançadas. Na tentativa de busca o servidor do Twitter reporta um erro e devolve um link. O link resposta funciona corretamente, mas o servidor não devolve os tweets requeridos. Isto deve-se a uma atualização do mecanismo de busca avançada dentro do Twitter que não existe mais e foi removida (/i/search/timeline) e que era utilizada pela API GetOldTweets.

FONTE: https://medium.com/@jcldinco/downloading-historical-tweets-using-tweet-ids-via-snscrape-and-tweepy-5f4ecbf19032